# Simple Permutation

In [1]:
import gc
import os
from math import exp
from collections import Counter
from typing import List, Optional, Union
from algorithm import PerplexityCalculator

import numpy as np
import pandas as pd
import transformers
import torch

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
PAD_TOKEN_LABEL_ID = torch.nn.CrossEntropyLoss().ignore_index
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
scorer = PerplexityCalculator("google/gemma-2-9b")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
samples = pd.read_csv("sample_submission.csv")

import numpy as np
BATCH_SIZE = 256

best_score = 1e6
import gc
import os
from math import exp
from collections import Counter
from typing import List, Optional, Union

import numpy as np
import pandas as pd
import transformers
import torch
import math


from functools import lru_cache

import random
import math
import random, pickle, warnings

samples.loc[0,"text"]='reindeer mistletoe elf gingerbread family advent scrooge chimney fireplace ornament'

samples.loc[1,"text"]='reindeer sleep walk the night and drive mistletoe scrooge laugh chimney jump elf bake gingerbread family give advent fireplace ornament'

samples.loc[2,"text"]='sleigh yuletide beard carol cheer chimney decorations gifts grinch holiday holly jingle magi naughty nice nutcracker ornament polar workshop stocking'

samples.loc[3,"text"]='sleigh of the magi yuletide cheer is unwrap gifts and eat cheer holiday decorations holly jingle relax sing carol visit workshop grinch naughty nice chimney stocking ornament nutcracker polar beard'

samples.loc[4,"text"]='from and of to the as in that it we with not you have milk chocolate candy peppermint eggnog cookie fruitcake toy doll game puzzle greeting card wrapping paper bow wreath poinsettia snowglobe candle fireplace wish dream hope believe wonder night star angel peace joy season merry hohoho kaggle workshop'
samples.loc[5,"text"]='from and and as we and have the in is it of not that the to with you advent card angel bake beard believe bow candy candle carol cheer cheer chocolate chimney cookie decorations doll dream drive eat eggnog family fireplace fireplace chimney fruitcake game gifts give gingerbread greeting grinch holiday holly hohoho hope jingle jump joy kaggle laugh magi merry milk mistletoe naughty nice night night elf nutcracker ornament ornament of the wrapping paper peace peppermint polar poinsettia puzzle reindeer relax scrooge season sing sleigh sleep snowglobe star stocking toy unwrap visit walk wish wonder workshop workshop wreath yuletide'

perplexities2 = []


for index, row in samples.iterrows(): # Step 1: Reorder the words based on POS tagging reordered_text = reorder_text(row["text"])
    score=scorer.get_perplexity(row['text'],batch_size=BATCH_SIZE)
    perplexities2.append(score)
    print(f"i={index}:{score}")
np.sum(perplexities2)/6

i=0:467.98558773246197
i=1:424.4444978122241
i=2:299.8028224298701
i=3:202.06587336346374
i=4:72.33079701870139
i=5:34.56939489431364


np.float64(250.19982887517247)

In [5]:
import itertools
def get_best_plex(df,ix, shift=0, rev=False, tf=False):
    print('SHIFT: ', shift, ' Score: ', df.loc[ix,'score'])
    for r in [ix]:
        if rev: shift+=1
        if len(df['text'][r].split(' ')) > shift:
            bGood = True
            while bGood == True:
                bGood = False
                t = df['text'][r].split(' ')
                if rev: shift *= -1
                last = [t[shift]]
                del t[shift]
                t = t + last
                best = df['score'][r]
                for x in range(len(t)-1):
                    new = t[:x] + last + t[x:-1]
                    new = ' '.join(new)
                    s = scorer.get_perplexity(new, batch_size=BATCH_SIZE)
                    if s < best:
                        bGood = True
                        best = s
                        df.at[r, 'score'] = s
                        df.at[r, 'text'] = new
                        print(r, x, "New Score: ", s)
                        print(new)
                        if tf: break #take first shift
    return df




def simulated_annealing_optimize_nb3(text: str, temp_start=6.0, temp_end=1.0, cooling_rate=0.2,
                                     n_neighbor=2, steps_per_temp=4, verbose=False, seq_to_choose=None,
                                     factor_acceptance=3,
                                     shuffle_start: bool = False, shuffle_pos=[], check_comb=True):


    words = text.split() ## first sample
    current = words[:]
    
    current_score = scorer.get_perplexity(' '.join(current), batch_size=BATCH_SIZE) ## calculate current perplexity
    
    ## checking combination
    if check_comb:
        res=comb_words.loc[comb_words["text"]==text]
        if len(res)>0:
            print(f"Combination has been looked before")
            return text,current_score
        
    while math.isnan(current_score):
        random.shuffle(current)
        current_score = scorer.get_perplexity(' '.join(current), batch_size=BATCH_SIZE)
        
    print(f"current={' '.join(current)}: {current_score}")
    
    ## shuffling initiate sample
    if shuffle_start:
        shuffle_pos = [i for i in shuffle_pos if i < len(current)]
        print(f"Shuffle positions: {shuffle_pos}")
        shuffled_values = random.sample([current[i] for i in shuffle_pos], len(shuffle_pos))
        for i, pos in enumerate(shuffle_pos):
            current[pos] = shuffled_values[i]
        current_score = scorer.get_perplexity(' '.join(current), batch_size=BATCH_SIZE)
        print(f"After shuffle: current={' '.join(current)}: {current_score}")
        
    best = current[:]
    best_score = current_score
    temp = temp_start

    seq_no_k = seq_to_choose if seq_to_choose is not None else list(range(len(words)))
    print(f"Seq to choose from: {seq_no_k}")
    
    ## simulated annealing
    while temp > temp_end:
        for _ in range(steps_per_temp):

            indices = random.sample(seq_no_k, min(n_neighbor, len(seq_no_k)))
            
            if n_neighbor == 2:
                neighbor = current[:]
                neighbor[indices[0]], neighbor[indices[1]] = neighbor[indices[1]], neighbor[indices[0]]
                neighbor_score = scorer.get_perplexity(' '.join(neighbor), batch_size=BATCH_SIZE)
                
            elif n_neighbor==3:
                neighbor = current[:]
                neighbor_variants = []
                for perm in [(1, 0, 2), (2, 0, 1), (0, 2, 1), (2, 1, 0), (1, 2, 0)]:
                    variant = current[:]
                    variant[indices[0]], variant[indices[1]], variant[indices[2]] = \
                        neighbor[indices[perm[0]]], neighbor[indices[perm[1]]], neighbor[indices[perm[2]]]
                    neighbor_variants.append(' '.join(variant))

                #perplexities = [scorer.get_perplexity(text, batch_size=BATCH_SIZE) for text in neighbor_variants]
                perplexities = scorer.get_perplexity(neighbor_variants, batch_size=BATCH_SIZE)
                best_variant_idx = perplexities.index(min(perplexities))
                neighbor = neighbor_variants[best_variant_idx].split()
                neighbor_score = scorer.get_perplexity(' '.join(neighbor), batch_size=BATCH_SIZE)
                
            else:
                print("n_neigbor must be either 2 or 3")
                return 0
            
            if math.isnan(neighbor_score):
                continue

            delta = neighbor_score - current_score
            acceptance_prob = math.exp(-delta / temp) * factor_acceptance
            if delta < 0 or random.random() < acceptance_prob:
                #print(f"acceptance_prob: {acceptance_prob}")
                current, current_score = neighbor[:], neighbor_score
                #if verbose: print(f"Temperature: {temp:.2f}, Current score:{current} {current_score:.2f}")
                #if verbose: print(f"Temperature: {temp:.2f}, Neigbor score: {neighbor_score:.2f}")
                if current_score < best_score:
                    best, best_score = current[:], current_score
                    if verbose>0:
                        print(f"New best: {' '.join(best)}")
                        print(f"New best score: {best_score:.2f}")

        temp -= cooling_rate
        
        if verbose>0:
            print(f"Temperature: {temp:.2f}, Current: {' '.join(current)}")
            print(f"Temperature: {temp:.2f}, Current score: {current_score:.2f}")


    if verbose:
        print(f"Final score: {best_score:.2f}")
    return ' '.join(best), best_score


def part_perm_brute_lastf(start_word, psize=3):
    """
    Efficiently finds the best permutation of the last part of the input string
    to minimize perplexity.
    """
    # Initial best word and score
    best_word = start_word
    best_score = scorer.get_perplexity(start_word, batch_size=BATCH_SIZE)
    print(f"Start score: {best_score}")

    # Split input into parts
    words = start_word.split(' ')
    rest = ' '.join(words[:psize]) + ' '
    part = words[psize:]

    import pandas as pd



    # Early exit if there are too many permutations
    if len(part) > 10:
        print("Too many permutations to process efficiently.")
        return start_word



    print(f"There are {math.factorial(len(part))} permutations.")
    # Iterate over permutations
    for i, perm in enumerate(itertools.permutations(part)):
        # Create the new word order
        new_word = rest + ' '.join(perm)
        #kombis.loc[i,"text"]=new_word

        # Calculate perplexity for the new word order
        new_score = scorer.get_perplexity(new_word, batch_size=BATCH_SIZE)

        #kombis.loc[i,"score"]=new_score
        # Log progress every 1000 iterations
        if i % 100 == 0:
            print(f"Iteration {i}: Current best score = {best_score}")
            #print(f"Tested sequence: {new_word} score: {new_score}")

        # Update best score and word if an improvement is found
        if new_score < best_score:
            print(f"New best score found: {new_score}")
            print(f"New best sequence: {new_word}")
            best_score = new_score
            best_word = new_word


    return best_word, best_score

In [6]:
##collection poor starting points

texts = [
    'wreath wish bow doll hope joy peace dream believe wonder star candle night with in and the have of it from as to not you we that poinsettia snowglobe angel fruitcake cookie peppermint candy greeting card fireplace kaggle puzzle game season merry hohoho chocolate milk eggnog toy workshop wrapping paper',
    'merry hohoho the season of eggnog and fruitcake greeting card kaggle workshop from with to in as that we you it have not toy doll puzzle game night chocolate milk cookie peppermint candy candle wreath snowglobe fireplace poinsettia wrapping paper bow angel star wish dream wonder believe hope joy peace'
]
comb_words=pd.DataFrame({'text':['']*(len(texts)+10), 'score':[1000.0]*(len(texts)+10)})
max_text=len(texts)
for idx, t in enumerate(texts):
    comb_words.loc[idx, "text"] = t


for ix in range(len(texts)):
    comb_words.loc[ix,"score"]=scorer.get_perplexity(comb_words.loc[ix,"text"], batch_size=BATCH_SIZE)
#text1=text
#comb_words.loc[comb_words["text"]==text1]
#comb_words.loc[0,"score"]=np.round(score,5)
comb_words.loc[:len(texts),:]

,text,score
0,wreath wish bow doll hope joy peace dream beli...,186.151806
1,merry hohoho the season of eggnog and fruitcak...,118.788399
2,,1000.000000


In [7]:
##initial solution
txt1=samples.loc[4,"text"]
scorer.get_perplexity(txt1,batch_size=512)

72.33079701870139

In [8]:
## sample 4
ii=4

## setting seq_to_choose to list(range(21,50)) 
#txt1=samples.loc[ii,"text"]
temp_text, temp_score=simulated_annealing_optimize_nb3(text=txt1, temp_start=6.0,
                                                    temp_end=1.0, cooling_rate=0.2,
                                n_neighbor=3,
                                  steps_per_temp=5, verbose=2, seq_to_choose=list(range(22,50)),
                                                           shuffle_start=False,
                                shuffle_pos=range(21,50), factor_acceptance=0.15,check_komb=True)

current=from and of to the as in that it we with not you have milk chocolate candy peppermint eggnog cookie fruitcake toy doll game puzzle greeting card wrapping paper bow wreath poinsettia snowglobe candle fireplace wish dream hope believe wonder night star angel peace joy season merry hohoho kaggle workshop: 72.33079701870139
Seq to choose from: [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
Temperature: 5.80, Current: from and of to the as in that it we with not you have milk chocolate candy peppermint eggnog cookie fruitcake toy doll game puzzle greeting card wrapping paper bow wreath poinsettia snowglobe candle fireplace wish dream hope believe wonder night star angel peace joy season merry hohoho kaggle workshop
Temperature: 5.80, Current score: 72.33
Temperature: 5.60, Current: from and of to the as in that it we with not you have milk chocolate candy peppermint eggnog cookie fruitcake toy doll game puzzle greetin

In [11]:
txt1 = "jingle yuletide carol sing unwrap gifts relax eat holiday cheer cheer decorations ornament holly stocking naughty and nice chimney sleigh workshop visit of the magi nutcracker grinch polar is beard"

ii=3
samples_new=samples.copy()
samples_new.loc[ii,"text"]=txt1
samples_new['score'] = samples_new['text'].map(lambda x: scorer.get_perplexity(x, batch_size=BATCH_SIZE))
for sh in range(6,10):
    df = get_best_plex(df=samples_new, ix=ii, shift=sh, rev=True, tf=True)

SHIFT:  6  Score:  258.44618746623325
SHIFT:  7  Score:  258.44618746623325
SHIFT:  8  Score:  258.44618746623325
SHIFT:  9  Score:  258.44618746623325


In [10]:
def find_best_reorder(input_str, index):
    words = input_str.split()
    if index < 0 or index >= len(words):
        raise ValueError("Index is out of range")

    best_sentence = input_str
    best_perplexity = scorer.get_perplexity(input_str, 4)
    #print(f"Start perplexity: {best_perplexity}, for sentence: {input_str}")

    word_to_move = words.pop(index)

    for i in range(len(words) + 1):
        #print(i)
        reordered_words = words[:]
        reordered_words.insert(i, word_to_move)
        reordered_sentence = ' '.join(reordered_words)

        #print(reordered_sentence)
        
        perplexity = scorer.get_perplexity(reordered_sentence, 4)

        if perplexity < best_perplexity:
            print(f"Checking permutation: {reordered_sentence} :with perplexity: {perplexity}")

def optimize_sentence(input_str):
    words = input_str.split()
    best_sentence = input_str
    best_perplexity = scorer.get_perplexity(input_str, 4)
    print(f"Initial perplexity: {best_perplexity}, for sentence: {input_str}")

    for index in range(len(words)):
        print(f"Index: {index}")
        find_best_reorder(best_sentence, index)

In [13]:
text_new = "ornament yuletide is the unwrap of holiday decorations gifts eat relax cheer cheer carol sing holly jingle sleigh workshop naughty and nice chimney stocking nutcracker magi visit grinch polar beard"
optimize_sentence(text_new)

Initial perplexity: 258.44618746623325, for sentence: ornament yuletide is the unwrap of holiday decorations gifts eat relax cheer cheer carol sing holly jingle sleigh workshop naughty and nice chimney stocking nutcracker magi visit grinch polar beard
Index: 0
Index: 1
Index: 2
Index: 3
Index: 4
Index: 5
Index: 6
Index: 7
Index: 8
Index: 9
Index: 10
Index: 11
Index: 12
Index: 13
Index: 14
Index: 15
Index: 16
Index: 17
Index: 18
Index: 19
Index: 20
Index: 21
Index: 22
Index: 23
Index: 24
Index: 25
Index: 26
Index: 27
Index: 28
Index: 29


# Multi solution ensemble

## imports

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import List, Dict, Tuple
import glob
import os
from tabulate import tabulate
from algorithm import PerplexityCalculator

## define

In [5]:
def load_submissions(base_dir: str = '.') -> List[Tuple[str, pd.DataFrame]]:
    """Load all submission files recursively from the directory."""
    submission_files = []
    
    # Walk through all directories recursively
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.csv') and 'submission' in file.lower():
                full_path = os.path.join(root, file)
                submission_files.append(full_path)
    
    if not submission_files:
        raise ValueError("No submission files found!")
    
    submissions = []
    print(f"\nLoading {len(submission_files)} submission files:")
    print("-" * 50)
    
    for file_path in submission_files:
        try:
            df = pd.read_csv(file_path)
            if 'id' in df.columns and 'text' in df.columns:
                print(f"Loading: {file_path}")
                submissions.append((file_path, df))
            else:
                print(f"Skipping {file_path} - Not a valid submission file (missing required columns)")
        except Exception as e:
            print(f"Error loading {file_path}: {str(e)}")
    
    if not submissions:
        raise ValueError("No valid submission files found!")
    
    return submissions

def evaluate_submissions(submissions: List[Tuple[str, pd.DataFrame]], 
                       scorer: PerplexityCalculator) -> Tuple[pd.DataFrame, Dict]:
    """Evaluate all submissions and track scores per text."""
    
    # Initialize storage for scores
    num_rows = len(submissions[0][1])  # Use the first submission's dataframe
    num_submissions = len(submissions)
    scores_matrix = np.zeros((num_rows, num_submissions))
    texts_matrix = [['' for _ in range(num_submissions)] for _ in range(num_rows)]
    
    print("\nEvaluating submissions:")
    print("-" * 50)
    
    submission_names = []
    
    # Calculate scores for each text in each submission
    for sub_idx, (file_path, submission) in enumerate(submissions):
        sub_name = file_path #os.path.basename(file_path)
        submission_names.append(sub_name)
        sub_scores = []
        
        print(f"\nSubmission {sub_idx + 1}/{num_submissions}: {sub_name}")
        
        for row_idx, text in enumerate(submission['text']):
            score = scorer.calculate_perplexity(text)
            scores_matrix[row_idx, sub_idx] = score
            texts_matrix[row_idx][sub_idx] = text
            sub_scores.append(score)
            print(f"Row {row_idx + 1}: Score = {score:.2f}")
        
        avg_score = np.mean(sub_scores)
        print(f"Average submission score: {avg_score:.2f}")
    
    # Create detailed scores DataFrame
    scores_df = pd.DataFrame(scores_matrix, columns=submission_names)
    
    # Find best texts and their sources
    best_texts = []
    best_scores = []
    best_sources = []
    
    for row_idx in range(num_rows):
        best_sub_idx = np.argmin(scores_matrix[row_idx])
        best_texts.append(texts_matrix[row_idx][best_sub_idx])
        best_scores.append(scores_matrix[row_idx, best_sub_idx])
        best_sources.append(submission_names[best_sub_idx])
    
    # Create summary dictionary
    summary = {
        'best_texts': best_texts,
        'best_scores': best_scores,
        'best_sources': best_sources,
        'submission_names': submission_names,
        'scores_matrix': scores_matrix
    }
    
    return scores_df, summary

def create_ensemble_submission(original_df: pd.DataFrame, 
                             summary: Dict, 
                             output_file: str):
    """Create final submission using best texts."""
    result_df = original_df.copy()
    result_df['text'] = summary['best_texts']
    
    # Print detailed results
    print("\nFinal Ensemble Results:")
    print("-" * 50)
    
    # Print submission averages first
    print("\nSubmission Averages:")
    for sub_idx, sub_name in enumerate(summary['submission_names']):
        avg_score = np.mean(summary['scores_matrix'][:, sub_idx])
        print(f"{sub_name}: {avg_score:.2f}")
    
    print("\nBest Texts Selected:")
    table_data = []
    for idx, (text, score, source) in enumerate(zip(
        summary['best_texts'], 
        summary['best_scores'], 
        summary['best_sources']
    )):
        table_data.append([
            idx,
            f"{text[:50]}..." if len(text) > 50 else text,
            f"{score:.2f}",
            source
        ])
    
    print(tabulate(table_data, 
                  headers=['Row', 'Best Text', 'Score', 'Source'],
                  tablefmt='grid'))
    
    final_avg_score = np.mean(summary['best_scores'])
    print(f"\nFinal Ensemble Average Score: {final_avg_score:.2f}")
    
    # Count usage of each submission
    source_counts = pd.Series(summary['best_sources']).value_counts()
    print("\nSubmission Usage Counts:")
    for source, count in source_counts.items():
        print(f"{source}: {count} texts")
    
    # Save final submission
    result_df.to_csv(output_file, index=False)
    print(f"\nEnsemble submission saved to {output_file}")
    
    # Also save detailed analysis
    analysis_file = 'submission_ensemble_analysis.csv'
    scores_df = pd.DataFrame({
        'Row': range(len(summary['best_texts'])),
        'Selected_Text': summary['best_texts'],
        'Score': summary['best_scores'],
        'Source': summary['best_sources']
    })
    scores_df.to_csv(analysis_file, index=False)
    print(f"Detailed analysis saved to {analysis_file}")

def main():
    # Initialize scorer
    scorer = PerplexityCalculator("google/gemma-2-9b")
    
    # Load all submission files from current directory and subdirectories
    submissions = load_submissions('/kaggle/input')
    
    # Evaluate all submissions
    scores_df, summary = evaluate_submissions(submissions, scorer)
    
    # Save detailed scores
    scores_df.to_csv('submission_scores_analysis.csv')
    print("\nDetailed scores saved to submission_scores_analysis.csv")
    
    # Create and save ensemble submission
    create_ensemble_submission(submissions[0][1], summary, 'submission.csv')

In [6]:
if __name__ == "__main__":
    main()

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

ValueError: No submission files found!